In [7]:
!pip install Pillow opencv-python numpy tensorflow

   ---------------------------------------- 0.0/385.0 MB ? eta -:--:--
   ---------------------------------------- 4.7/385.0 MB 25.9 MB/s eta 0:00:15
   - -------------------------------------- 9.7/385.0 MB 24.1 MB/s eta 0:00:16
   - -------------------------------------- 13.6/385.0 MB 23.7 MB/s eta 0:00:16
   - -------------------------------------- 17.0/385.0 MB 21.1 MB/s eta 0:00:18
   -- ------------------------------------- 20.4/385.0 MB 19.9 MB/s eta 0:00:19
   -- ------------------------------------- 23.9/385.0 MB 19.1 MB/s eta 0:00:19
   -- ------------------------------------- 27.3/385.0 MB 18.8 MB/s eta 0:00:20
   --- ------------------------------------ 30.9/385.0 MB 18.7 MB/s eta 0:00:19
   --- ------------------------------------ 36.2/385.0 MB 19.5 MB/s eta 0:00:18
   ---- ----------------------------------- 41.2/385.0 MB 20.0 MB/s eta 0:00:18
   ---- ----------------------------------- 45.6/385.0 MB 20.2 MB/s eta 0:00:17
   ----- ---------------------------------- 50.1/38

DEPRECATION: Loading egg at c:\python311\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
ERROR: Could not install packages due to an OSError: [WinError 2] The system cannot find the file specified: 'C:\\Python311\\Scripts\\markdown_py.exe' -> 'C:\\Python311\\Scripts\\markdown_py.exe.deleteme'



# Crop and resize images

The images are 3552x3552 pixels and contain a lot of empty space at the edges. Here we crop the images to 2048x2048 toward the center. Then they are resized to 64x64.

In [1]:
from PIL import Image
import os
from pprint import pprint

dirs = [ (f'.\\data\\raw_data\\{num}', f'.\\data\\processed_64\\{num}') for num in range(16) ]
# dirs = [("./data/sanity_check_raw", "./data/sanity_check")]

pprint(dirs)
for (in_dir, out_dir) in dirs:
    os.makedirs(out_dir, exist_ok=True)

original_size = 3552
crop_size = 2048
target_size = 64
max_files = 60 # We want the same number of images of each ball

[('.\\data\\raw_data\\0', '.\\data\\processed_64\\0'),
 ('.\\data\\raw_data\\1', '.\\data\\processed_64\\1'),
 ('.\\data\\raw_data\\2', '.\\data\\processed_64\\2'),
 ('.\\data\\raw_data\\3', '.\\data\\processed_64\\3'),
 ('.\\data\\raw_data\\4', '.\\data\\processed_64\\4'),
 ('.\\data\\raw_data\\5', '.\\data\\processed_64\\5'),
 ('.\\data\\raw_data\\6', '.\\data\\processed_64\\6'),
 ('.\\data\\raw_data\\7', '.\\data\\processed_64\\7'),
 ('.\\data\\raw_data\\8', '.\\data\\processed_64\\8'),
 ('.\\data\\raw_data\\9', '.\\data\\processed_64\\9'),
 ('.\\data\\raw_data\\10', '.\\data\\processed_64\\10'),
 ('.\\data\\raw_data\\11', '.\\data\\processed_64\\11'),
 ('.\\data\\raw_data\\12', '.\\data\\processed_64\\12'),
 ('.\\data\\raw_data\\13', '.\\data\\processed_64\\13'),
 ('.\\data\\raw_data\\14', '.\\data\\processed_64\\14'),
 ('.\\data\\raw_data\\15', '.\\data\\processed_64\\15')]


In [2]:
left = (original_size - crop_size) // 2
top = (original_size - crop_size) // 2
right = (original_size + crop_size) // 2
bottom = (original_size + crop_size) // 2

# Values for phone pictures
# top = 200
# left = 360
# bottom = 760
# right = 820

for (in_dir, out_dir) in dirs:
    files = os.listdir(in_dir)
    print(f"Processing {in_dir} ...")

    counter = 0
    for filename in files[:max_files]:
        if filename.lower().endswith(".jpg"):
            counter += 1
            # Open the image
            img_path = os.path.join(in_dir, filename)
            output_path = os.path.join(out_dir, filename)
            
            Image.open(img_path
                ).crop((left, top, right, bottom)
                ).resize((target_size, target_size), Image.Resampling.LANCZOS
                ).save(output_path)

            # print(f"Cropped and saved: {output_path} ({counter} of {max_files})")

Processing .\data\raw_data\0 ...
Processing .\data\raw_data\1 ...
Processing .\data\raw_data\2 ...
Processing .\data\raw_data\3 ...
Processing .\data\raw_data\4 ...
Processing .\data\raw_data\5 ...
Processing .\data\raw_data\6 ...
Processing .\data\raw_data\7 ...
Processing .\data\raw_data\8 ...
Processing .\data\raw_data\9 ...
Processing .\data\raw_data\10 ...
Processing .\data\raw_data\11 ...
Processing .\data\raw_data\12 ...
Processing .\data\raw_data\13 ...
Processing .\data\raw_data\14 ...
Processing .\data\raw_data\15 ...


# Split the data into training, validation and testing datasets

In [66]:
import pathlib
import numpy as np
import keras
from sklearn.model_selection import train_test_split

data_dir = './data/processed_64'
batch_size = 25
img_width = 64
img_height = 64

dataset = keras.utils.image_dataset_from_directory(
    data_dir,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    label_mode='int'
)

class_names = dataset.class_names # ['0', '1', '10', '11', '12', '13', '14', '15', '2', '3', '4', '5', '6', '7', '8', '9']

image_batches = []
label_batches = []

for images, labels in dataset:
    image_batches.append(images)
    label_batches.append(labels)

X = np.concatenate(image_batches)
y = np.concatenate(label_batches)

print("Splitting data into training, validation and testing datasets...")

X_train, X_rest, y_train, y_rest = train_test_split(X, y, test_size = 0.25, random_state = 0)

X_val, X_test, y_val, y_test = train_test_split(X_rest, y_rest, test_size = (2/3), random_state = 0)

print("X_train", X_train.shape)
print("y_train", y_train.shape)
print("X_val", X_val.shape)
print("y_val", y_val.shape)
print("X_test", X_test.shape)
print("y_test", y_test.shape)

Found 960 files belonging to 16 classes.
Splitting data into training, validation and testing datasets...
X_train (720, 64, 64, 3)
y_train (720,)
X_val (80, 64, 64, 3)
y_val (80,)
X_test (160, 64, 64, 3)
y_test (160,)


In [67]:
import tensorflow as tf

train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train))
val_ds = tf.data.Dataset.from_tensor_slices((X_val, y_val))
test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test))

# Data augmentation

In [68]:
from keras import layers
import tensorflow as tf
import numpy as np

normalization_layer = keras.layers.Rescaling(1./255) # Rescale RGB values from 0..255 to floats in 0..1

augmentation_layer = keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.2),
    layers.RandomBrightness(0.05)
])

batch_size = 50
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.map(
    lambda x, y: (augmentation_layer(x), y)).map(
    lambda x, y: (normalization_layer(x), y)
    ).shuffle(buffer_size=len(X_train), seed=0, reshuffle_each_iteration=True
    ).batch(batch_size
    ).prefetch(buffer_size=AUTOTUNE)

val_ds = val_ds.map(
    lambda x, y: (normalization_layer(x), y)
    ).batch(batch_size
    ).prefetch(buffer_size=AUTOTUNE)

test_ds = test_ds.map(
    lambda x, y: (normalization_layer(x), y)
    ).batch(batch_size
    ).prefetch(buffer_size=AUTOTUNE)

# Create and train the CNN

In [69]:
num_classes = 16
model = keras.Sequential([
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

model.compile(optimizer="adam", loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=["accuracy"])

In [70]:
epochs = 50
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs = epochs
)

Epoch 1/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 2s 23ms/step - accuracy: 0.0568 - loss: 2.7882 - val_accuracy: 0.1125 - val_loss: 2.7291
Epoch 2/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.1529 - loss: 2.6524 - val_accuracy: 0.2375 - val_loss: 2.2633
Epoch 3/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.2825 - loss: 2.1051 - val_accuracy: 0.5125 - val_loss: 1.4607
Epoch 4/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.5283 - loss: 1.4026 - val_accuracy: 0.7250 - val_loss: 0.9527
Epoch 5/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.6053 - loss: 1.1428 - val_accuracy: 0.8000 - val_loss: 0.7184
Epoch 6/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.7302 - loss: 0.7818 - val_accuracy: 0.6250 - val_loss: 0.8014
Epoch 7/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.7148 - loss: 0.6968 - val_accuracy: 0.8000 - val_loss: 0.4975
Epoch 8/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.8030 - loss: 0.5265 - val_accuracy: 0.8750 - v

In [71]:
loss, acc = model.evaluate(test_ds)
print("Loss: ", loss, "\n", "Accuracy: ", acc)

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9727 - loss: 0.0721 
Loss:  0.06632108986377716 
 Accuracy:  0.9750000238418579


In [73]:
image = keras.utils.load_img(".\\data\\sanity_check\\14.JPG")

image_array = keras.utils.img_to_array(image)
image_array = tf.expand_dims(image_array, 0) # Create a batch

predictions = model.predict(image_array)
score = tf.nn.softmax(predictions[0])

print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
This image most likely belongs to 14 with a 100.00 percent confidence.
